## Sentinel-2 crop

In [92]:
service = dict([('title', 'S2 Crop and GeoTiff conversion'),
                ('abstract', 'Sentinel-2 crop and GeoTiff conversion'),
                ('id', 's2_crop')])

In [93]:
endpoint = dict([('id', 'endpoint'),
               ('value', 'https://catalog.terradue.com/sentinel2/search'),
               ('title', 'Catalogue endpoint URL'),
               ('abstract', 'Catalogue endpoint URL for dataset selection')])

In [94]:
start = dict([('id', 'start'),
              ('value', '2018-05-05T10:00:00Z'),
              ('title', 'start date for dataset selection'),
              ('abstract', 'start date for dataset selection')])

In [95]:
stop = dict([('id', 'stop'),
            ('value', '2018-05-05T23:59:59Z'),
            ('title', 'stop date for dataset selection'),
            ('abstract', 'stop date for dataset selection')])

In [96]:
search_type = dict([('id', 'time_filter_type'),
            ('value', 'date'),
            ('title', 'search time filter type'),
            ('abstract', 'search time filter type for dataset selection (date|update)')])

In [97]:
cat_index = dict([('id', 'cat_index'),
              ('value', 'ecop-cnr-issia'),
              ('title', 'publishing catalog index'),
              ('abstract', 'publishing catalog index')])

In [98]:
config_url = dict([('id', 'config_url'),
                   ('value', 'https://store.terradue.com/ec-ecopotential/config/config.ini'),
                   ('title', 'Configuration file URL'),
                   ('abstract', 'Configuration file URL')])

In [99]:
username = dict([('id', 'username'),
                 ('value', 'ecop-cnr-issia'),
                 ('title', 'username for DA access'),
                 ('abstract', 'username for DA access')])

In [100]:
api_key  = dict([('id', 'api_key'),
                 ('value', 'AKCp5aUjBi3JvjztBDdqPMMjM9beDhtcMLjGsn5axpviB3rAndimLeqfK3bTqUJsR2MtAkjiY'),
                 ('title', 'apikey for DA access'),
                 ('abstract', 'apikey for DA access')])

In [101]:
data_path = '/workspace/data'

In [102]:
input_reference = "camargue"

In [103]:
input_identifier = 'S2B_MSIL1C_20180609T103019_N0206_R108_T31TEJ_20180609T131037'
#input_identifier = 'S2A_MSIL1C_20170909T110651_N0205_R137_T30STF_20170909T111217'
#the last one actual area has no intersection with the AOI
#input_identifier = 'S2A_MSIL1C_20180505T103021_N0206_R108_T31TFH_20180505T124726'


In [104]:
input_config_url = 'https://store.terradue.com/ec-ecopotential/config/config.ini'

In [105]:
import requests
import StringIO
import configparser
import warnings
warnings.filterwarnings("ignore")

try:
    r = requests.get(input_config_url, headers={"X-JFrog-Art-Api":api_key['value'], 'User-Agent': 'curl/t2Client'})

    ini_content = ''

    if r.status_code == 200:
        ini_content = r.content

    if not ini_content:
        raise ValueError('No ini_content')

        # read the configuration values
    buf = StringIO.StringIO(ini_content)
    config = configparser.ConfigParser()
    config.readfp(buf)
except Exception as e:
    raise ValueError('Could not get config.ini : %s' %e)

In [106]:
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import snappy
import dateutil.parser as parser
import gc
from datetime import datetime
import matplotlib
import matplotlib.colors as colors

from os.path import basename
import gdal
import osr
import zipfile

import subprocess

%matplotlib inline

In [107]:
import sys

if sys.path.count('/application/notebook/libexec/') == 0:
    sys.path.append('/application/notebook/libexec/')
sys.path.append(os.getcwd())


### Get configuration

In [108]:
ftp_server = str(config.get('ftp', 'hostname'))
base_path=str(config.get('ftp', 'base_path'))

input_ftp_server_address = 'sftp://%s%s' %(ftp_server,base_path)

pa_code = str(config.get(input_reference, 'pa_code'))
pa_name = str(config.get(input_reference, 'pa_name'))
#pa_bbox = str(config.get(input_reference, 'pa_bbox'))

shapefile_url = str(config.get(input_reference, 'shapefile_url'))
shapefile_url_crop = str(config.get(input_reference, 'shapefile_url_crop'))

percentage_threshold = str(config.get(input_reference, 'percentage_threshold'))

In [109]:
s2prd = "%s/%s/%s.SAFE/MTD_MSIL1C.xml" % (data_path, input_identifier, input_identifier)
product = snappy.ProductIO.readProduct(s2prd)

width = product.getSceneRasterWidth()
height = product.getSceneRasterHeight()
name = product.getName()
description = product.getDescription()
band_names = product.getBandNames()

In [110]:
product_date = parser.parse(product.getStartTime().toString()).date()

In [111]:
product_date = parser.parse(product.getStartTime().toString()).date()
START_DATE = '%sZ' %parser.parse(product.getStartTime().toString()).isoformat()
END_DATE = '%sZ' %parser.parse(product.getEndTime().toString()).isoformat()

now = datetime.now()

metadata_date = now.strftime("%Y-%m-%d")
metadata_year = now.strftime("%Y")

creation_date = START_DATE

date_path = '%s/%02d/%02d' % (product_date.year, product_date.month, product_date.day)
    
middle_path = '%s/EO_Data/Sentinel2/Raw'%pa_name
        
ftp_remote_path = os.path.join(input_ftp_server_address, date_path)
zip_output_name = '%s_PA_%s_CROP.zip' % (name, pa_name)
download_URL = '%s/%s' %(ftp_remote_path, zip_output_name)

### Getting Shapefiles from store

In [112]:
import shapefile
from shapely.geometry import shape

try:
    for shpf_url in [shapefile_url,shapefile_url_crop]:
        
        for ext in ['prj','shp', 'shx', 'dbf', 'sbx', 'sbn']:
        
            shp_url = '%s.%s' %(shpf_url,ext)
            r = requests.get(shp_url,headers={"X-JFrog-Art-Api":api_key['value'], 'User-Agent': 'curl/t2Client'})
            
            if r.status_code == 200 and r.content:
                shapein = os.path.join(os.path.join(data_path,os.path.basename(shp_url)))
                
                with open(shapein,"wb") as f:
                    for chunk in r.iter_content(chunk_size=1024):
                        # writing one chunk at a time to pdf file
                        if chunk:
                            f.write(chunk)
                    f.close()
            else:
                raise ValueError
except Exception as e:
    raise

shape_shp = os.path.join(os.path.join(data_path,os.path.basename('%s.shp' %shapefile_url)))
shape_shp_crop = os.path.join(os.path.join(data_path,os.path.basename('%s.shp' %shapefile_url_crop)))

shapeIN = shapefile.Reader(shape_shp)
feature = shapeIN.shapeRecords()[0]
first = feature.shape.__geo_interface__  

shp_geom = shape(first)


### Cloud coverage analysis over the cropped area

In [113]:
snappy.GPF.getDefaultInstance().getOperatorSpiRegistry().loadOperatorSpis()

HashMap = snappy.jpy.get_type('java.util.HashMap')

parameters = HashMap()
parameters.put('referenceBand', 'B1')

resampleB1 = snappy.GPF.createProduct('Resample', parameters, product)

In [114]:
HashMap = snappy.jpy.get_type('java.util.HashMap')

BandDescriptor = snappy.jpy.get_type('org.esa.snap.core.gpf.common.BandMathsOp$BandDescriptor')

targetBand0 = BandDescriptor()
targetBand0.name = 'cloud_mask'
targetBand0.type = 'uint16'
targetBand0.expression = 'opaque_clouds_60m'


targetBands = snappy.jpy.array('org.esa.snap.core.gpf.common.BandMathsOp$BandDescriptor', 1)
targetBands[0] = targetBand0

 
parameters = HashMap()
parameters.put('targetBands', targetBands)

cloud_mask = snappy.GPF.createProduct('BandMaths', parameters, resampleB1)

In [115]:
WKTReader = snappy.jpy.get_type('com.vividsolutions.jts.io.WKTReader')

geom = WKTReader().read(str(shp_geom))


parameters = HashMap()
parameters.put('copyMetadata', True)
parameters.put('geoRegion', geom)
    
try:
    cloud_mask_geo = snappy.GPF.createProduct('Subset', parameters, cloud_mask)
except:
    pass
finally:
    cmg_bands = cloud_mask_geo.getNumBands()
    cloud_percent = 100.0

#if cmg_bands == 0:
#    raise ValueError('NO intersection between AOI and actual dataset area') 
    
if cmg_bands != 0:
    mask_geo_output_name = '%s_%s_MASK_%s.tif' % (name, pa_code, '60')
    snappy.ProductIO.writeProduct(cloud_mask_geo, mask_geo_output_name, 'GeoTIFF')


In [116]:
if cmg_bands != 0:
    import gdalnumeric
    raster_file = gdalnumeric.LoadFile(mask_geo_output_name)
    #print raster_file.min(), raster_file.max()
    pixel_count_cloud_geo = (raster_file == 255).sum()  # for pixel value = 1
    #print pixel_count_cloud_geo
    cloud_percent =  float(pixel_count_cloud_geo) / float(raster_file.size) * 100.0
    #print 'cloud percent over cropped area: %s'%cloud_percent

In [117]:
if cmg_bands != 0:
    mask_output_name = '%s_MASK_%s.tif' % (name, '60')
    snappy.ProductIO.writeProduct(cloud_mask, mask_output_name, 'GeoTIFF')
    import gdalnumeric

    raster_file = gdalnumeric.LoadFile(mask_output_name)

    pixel_count_cloud = (raster_file == 255).sum()  # for pixel value = 1

    cloud_percent_mask =  float(pixel_count_cloud) / float(raster_file.size) * 100.0
    #print 'cloud percent over entire area: %s'%cloud_percent_mask
    
    loud_mask_geo = None
    raster_file = None

    gc.collect()

### Crop

In [118]:
if cloud_percent <= percentage_threshold and cmg_bands != 0:
    
    import shlex
    import re
    
    bands_ref_list = ['B01', 'B02','B03','B04','B05','B06','B07','B08','B8A','B09','B10','B11','B12']
    base_bands_path = '%s/%s/%s.SAFE/GRANULE/' % (data_path, input_identifier, input_identifier)
    
    pattern = re.compile(r'_B.*\.jp2')

    dataset_name = os.listdir(base_bands_path)
    
    bands_path = '%s%s/IMG_DATA' % (base_bands_path, dataset_name[0])

    jp2_files = filter(lambda x: re.findall(pattern,x), os.listdir(bands_path))
    n_files_orig = len(jp2_files)
    jp2_basename = jp2_files[0].split('_B')[0]

    jp2_files=dict()
    for b in bands_ref_list:
        jp2_files[b] = '%s_%s.jp2'%(jp2_basename,b)
    n_files_new = len(jp2_files)
    
    assert (n_files_orig == n_files_new) , "Cropping phase: Number of bands not corresponding to the expected ones"
    
    geotiffs = dict()
    
    for b_name in bands_ref_list:

        output_name = '%s_PA_%s_CROP_%s.tif' % (name, pa_name, b_name)
        
        band_in = '%s/%s' %(bands_path,jp2_files[b_name])
        
        #-tr 60.0 60.0 
        cmd='/opt/anaconda/bin/gdalwarp -q -cutline %s -crop_to_cutline -co COMPRESS=LZW -of GTiff %s %s' %(shape_shp_crop, band_in, output_name)
        
        args = shlex.split(cmd)

        try:
            p = subprocess.Popen(args,
                                 stdout=subprocess.PIPE,
                                 stdin=subprocess.PIPE,
                                 stderr=subprocess.PIPE)
        except Exception, e:
            #print 'An error occured while clippind data: %s' %e
            raise Exception('An error occured while clippind data %s: \n%s' %(output_name_tmp,e))

        res, err = p.communicate()
        if p.returncode:
            raise Exception('Subprocess returned: %s \n %s \n on command %s' %(str(p.returncode), err, cmd))
            
        geotiffs[b_name] = output_name

In [119]:
%matplotlib inline

def plotTiff(filename):

    data = gdal.Open(filename)
    bnd1 = data.GetRasterBand(1)

    img1 = bnd1.ReadAsArray(0,0,data.RasterXSize, data.RasterYSize)
    
    imgplot = plt.imshow(img1, cmap=plt.cm.binary_r)    
    
    return imgplot

In [120]:
%matplotlib inline

def plotBand(product, band, vmin, vmax):
    
    band = product.getBand(band)

    w = band.getRasterWidth()
    h = band.getRasterHeight()
    
    band_data = np.zeros(w * h, np.float32)
    band.readPixels(0, 0, w, h, band_data)

    band_data.shape = h, w

    imgplot = plt.imshow(band_data, cmap=plt.cm.binary_r, vmin=vmin, vmax=vmax)

    
    return imgplot 


In [121]:
if False and cmg_bands != 0:
    fig = plt.figure(figsize=(20,20))
    
    for index, geotiff in enumerate(geotiffs):
        a=fig.add_subplot(4,4,index+1)
        imgplot = plotTiff(geotiff)
        nameB = band_names[index]
        a.set_title('%s %s-cropped' %(nameB, pa_name))
    
    a=fig.add_subplot(4,4,index+1)
    imgplot = plotTiff(mask_geo_output_name)
    a.set_title('cloud mask cropped')
    
    a=fig.add_subplot(4,4,index+2)
    imgplot = plotTiff(mask_output_name)
    a.set_title('cloud mask')
    
    plt.tight_layout()
    fig = plt.gcf() 
    plt.show()

    fig.clf()
    plt.close()
    gc.collect()
    

In [122]:
def set_dates():
    
    iso_metadata.set_start_date(START_DATE)
    iso_metadata.set_end_date(END_DATE)

In [123]:
def set_info(short_name, title, abstract):
    
    iso_metadata.set_identifier(short_name)
    iso_metadata.set_title(title)
    iso_metadata.set_abstract(abstract)

In [124]:
def set_geo(band_name,geotiff):
    
    sp_i = dict()
    ds = gdal.Open(geotiff)

    iso_metadata.set_col_size(str(ds.RasterXSize))
    iso_metadata.set_row_size(str(ds.RasterYSize))
    sp_i['row_size'] = str(ds.RasterXSize)
    sp_i['col_size'] = str(ds.RasterYSize)
    
    transform = ds.GetGeoTransform()
    
    sp_i['row_res'] = str(transform[1])
    sp_i['col_res'] = str(transform[1])
    
    iso_metadata.set_pixel_size(str(transform[1]))

    ul_x = transform[0]
    ul_y = transform[3]

    nw_corner = '%s %s' % (str(ul_x), str(ul_y))
    sp_i['nw_corner'] = nw_corner

    iso_metadata.set_nw_corner(nw_corner)

    lr_x = transform[0] + transform[1] * ds.RasterXSize
    lr_y = transform[3] + transform[5] * ds.RasterYSize

    se_corner = '%s %s' % (str(lr_x), str(lr_y))
    sp_i['se_corner'] = se_corner
    
    iso_metadata.set_se_corner(se_corner)
        
    old_cs= osr.SpatialReference()
    old_cs.ImportFromWkt(ds.GetProjectionRef())

    wgs84_wkt = """
GEOGCS["WGS 84",
    DATUM["WGS_1984",
        SPHEROID["WGS 84",6378137,298.257223563,
            AUTHORITY["EPSG","7030"]],
        AUTHORITY["EPSG","6326"]],
    PRIMEM["Greenwich",0,
        AUTHORITY["EPSG","8901"]],
    UNIT["degree",0.01745329251994328,
        AUTHORITY["EPSG","9122"]],
    AUTHORITY["EPSG","4326"]]"""
    new_cs = osr.SpatialReference()
    new_cs .ImportFromWkt(wgs84_wkt)

    transform = osr.CoordinateTransformation(old_cs,new_cs) 

    min_lon = transform.TransformPoint(lr_x, ul_y)[0]
    iso_metadata.set_min_lon(str(min_lon))

    min_lat = transform.TransformPoint(ul_x,lr_y)[1]
    iso_metadata.set_min_lat(str(min_lat))

    max_lon = transform.TransformPoint(ul_x,lr_y)[0]
    iso_metadata.set_max_lon(str(max_lon))

    max_lat = transform.TransformPoint(lr_x, ul_y)[1]
    iso_metadata.set_max_lat(str(max_lat))
    
    sp_i['min_lon'] = str(min_lon)
    sp_i['min_lat'] = str(min_lat)
    sp_i['max_lon'] = str(max_lon)
    sp_i['max_lat'] = str(max_lat)
    
    prj = ds.GetProjection()
    srs=osr.SpatialReference(wkt=prj)

    sp_i['epsg_code'] = srs.GetAttrValue("PROJCS|AUTHORITY", 1)
    iso_metadata.set_epsg_code(srs.GetAttrValue("PROJCS|AUTHORITY", 1))
    
    # adding single band spatial info to the global spatial info dict for the general ISOmetadata
    spatial_info[band_name] = sp_i

In [125]:
def write_properties(properties_file):
    
    properties = open(properties_file + '.properties', 'w')

    properties.write('identifier=' + geotiff)
    properties.write('\n')
    properties.write('date=%s/%s' % (START_DATE, END_DATE))
    properties.write('\n')
    properties.write('category=%s,http://www.terradue.com/api/data-pipeline/results,%s Protected Area resource' % (pa_code, pa_name))
 
    properties.close()

In [128]:
import ISOMetadata

#print dir(ISOMetadata.ISOMetadata())

if cloud_percent <= percentage_threshold and cmg_bands != 0:
    iso_metadata = ISOMetadata.ISOMetadata()
    iso_metadata.set_contact('info@terradue.com')
    iso_metadata.set_date(metadata_date)
    iso_metadata.set_creation_date(creation_date)
    iso_metadata.set_data_format('GEOTIFF')
    iso_metadata.set_data_type('UInt16')
    iso_metadata.set_pa(pa_name)
    iso_metadata.set_data_quality('Nominal')
    iso_metadata.set_responsible_party('CNR')
    iso_metadata.set_onlineResource(download_URL)
    lineage_template = 'Terradue [%s], Copernicus Sentinel data [%s]. \
This work has received funding from the European Union\'s Horizon 2020 research and innovation programme \
under grant agreement No 641762 (ECOPOTENTIAL: improving future ecosystem benefits through earth observations).'
    lineage_str = lineage_template %(str(metadata_year), str(product_date.year))
    print lineage_str
    iso_metadata.set_lineage_template(lineage_str)
    

In [129]:
if cloud_percent <= percentage_threshold and cmg_bands != 0:

    spatial_info = dict()
    
    for b_name in bands_ref_list:
        
        geotiff = geotiffs[b_name]
        
        set_dates()
        
        short_name = os.path.splitext(basename(geotiff))[0]
        
        title = 'Sentinel 2 Level 1C Band %s' % b_name
        abstract = 'Sentinel 2 Level 1C Top of Atmosphere Reflectance for Band %s' % b_name
        
        set_info(short_name, title, abstract)

        set_geo(b_name, geotiff)

        iso_metadata.write(os.path.splitext(basename(geotiff))[0] + '.xml')

        write_properties(basename(geotiff))

### Zipping the cropped bands and related metadata

In [130]:
if cloud_percent <= percentage_threshold and cmg_bands != 0:  
    
    zip_output_name = '%s_PA_%s_CROP.zip' % (name, pa_name)
    zf = zipfile.ZipFile(zip_output_name, mode='w')
    try:
        for b_name in bands_ref_list:
        
            geotiff = geotiffs[b_name]
            
            short_name = os.path.splitext(basename(geotiff))[0]
            #print '%s %s.properties %s.xml' %(geotiff,geotiff,short_name)
            zf.write(geotiff)
            zf.write('%s.properties' %geotiff)
            zf.write('%s.xml' %short_name)
    finally:
        #print 'closing'
        zf.close()


### Writing General ISOMetadata file

In [131]:
if cloud_percent <= percentage_threshold and cmg_bands != 0:
    
    short_name = os.path.splitext(zip_output_name)[0]
    title = 'Sentinel 2 Level 1C %s (cropped Bands) ' % name
    abstract = 'Sentinel 2 Level 1C Top of Atmosphere Reflectance for 13 Bands cropped over %s' % pa_name
        
    set_info(short_name, title, abstract)
    
    iso_metadata.set_spatialReprInfo_elems(spatial_info, bands_ref_list)

    iso_metadata.set_spatial_resolutions([str(10),str(20),str(60)])
    
    iso_metadata.set_lineage_template(lineage_str)
    
    iso_metadata.set_min_lon(spatial_info['B01']['min_lon'])
    iso_metadata.set_min_lat(spatial_info['B01']['min_lat'])
    iso_metadata.set_max_lon(spatial_info['B01']['max_lon'])
    iso_metadata.set_max_lat(spatial_info['B01']['max_lat'])
    
    iso_metadata.write(os.path.splitext(zip_output_name)[0] + '.xml')
